In [1]:
## data manipulation
import pandas as pd
## show dataframe
from IPython.display import display
## Loading data from python to sql database
import sqlalchemy as sa
from sqlalchemy import create_engine
## working with mysql
import pymysql
## hiding MySQL credentials - ref: https://youtu.be/CJjSOzb0IYs
import mysqlcredentials

In [2]:
profile_info = pd.read_csv('datasets/profile_info_2022-09-19.csv')
profile_mentions = pd.read_csv('datasets/profile_mentions_2022-09-19.csv')
display(profile_info, profile_mentions.head(3))

,date,screen_name,followers,following,posts,lists,likes
0,2022-09-19,LulaOficial,4373145,2805,27058,4066,14991
1,2022-09-19,jairbolsonaro,8933479,497,14978,8415,5886
2,2022-09-19,cirogomes,1488679,415,12764,3278,5837
3,2022-09-19,simonetebetbr,366207,239,3069,853,12423


,start,start_date,start_time,end,end_date,end_time,@LulaOficial_mentions,@LulaOficial_mentions_without_retweet,@jairbolsonaro_mentions,@jairbolsonaro_mentions_without_retweet,@cirogomes_mentions,@cirogomes_mentions_without_retweet,@simonetebetbr_mentions,@simonetebetbr_mentions_without_retweet
0,2022-09-12 19:00:00-03:00,2022-09-12,19:00:00,2022-09-12 20:00:00-03:00,2022-09-12,20:00:00,12009,6035,7892,2065,3210,1538,253,180
1,2022-09-12 20:00:00-03:00,2022-09-12,20:00:00,2022-09-12 21:00:00-03:00,2022-09-12,21:00:00,22107,8869,6958,2129,2604,1082,237,164
2,2022-09-12 21:00:00-03:00,2022-09-12,21:00:00,2022-09-12 22:00:00-03:00,2022-09-12,22:00:00,23491,10706,7819,2431,3043,1685,358,227


In [3]:
# Conection to MySQL to upload data
user = mysqlcredentials.user
passw = mysqlcredentials.passw
host =  mysqlcredentials.host
database = mysqlcredentials.database
port = mysqlcredentials.port

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}"
                       .format(user=user,
                               pw=passw,
                               host=host,
                               port=port,
                               db=database))

In [4]:
## Filtering data to avoid duplicates
last_update = pd.read_sql("SELECT * FROM vw_last_update", engine)
try:
    new_profile_info = profile_info.loc[(pd.to_datetime(profile_info['date']).dt.date > pd.to_datetime(last_update.loc[0, 'end_date']).date())]

    new_profile_mentions = profile_mentions.loc[
        pd.to_datetime(profile_mentions['end']).dt.tz_localize(None) > pd.to_datetime(last_update.loc[0, 'end']).tz_localize(None)
        ].reset_index(drop=True)
        
    display(new_profile_info, new_profile_mentions)
except:
    if len(last_update) == 0:
        print('database is empty')

,date,screen_name,followers,following,posts,lists,likes


,start,start_date,start_time,end,end_date,end_time,@LulaOficial_mentions,@LulaOficial_mentions_without_retweet,@jairbolsonaro_mentions,@jairbolsonaro_mentions_without_retweet,@cirogomes_mentions,@cirogomes_mentions_without_retweet,@simonetebetbr_mentions,@simonetebetbr_mentions_without_retweet


In [5]:
new_profile_info.to_sql(con=engine, name='profile_info', if_exists='append', index=False)
new_profile_mentions.to_sql(con=engine, name='profile_mentions', if_exists='append', index=False)